# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [ ]:
!wget -cO Cat_Dog+data.zip https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip -n Cat_Dog_data.zip

In [11]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [12]:
print(len(trainloader))
trainloader.batch_size

352


64

In [13]:
data_dir = 'Lesson05/Cat_Dog_data'

# transform = transforms.Compose([transforms.Resize(255),
#                                  transforms.CenterCrop(224),
#                                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#                                  transforms.ToTensor()])
transform = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
# TODO: Define transforms for the training data and testing data
train_transforms = transform

test_transforms = transform

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)


We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [14]:
model = models.densenet121(pretrained=True)


In [15]:
model.classifier

Linear(in_features=1024, out_features=1000, bias=True)

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [16]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier
model.classifier

Sequential(
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (output): LogSoftmax()
)

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [17]:
import time

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

#Training parameters
epochs = 1
GD_steps = 0
print_every = 5  #GD_steps

train_losses, test_losses, accuracy_values = [], [], []

for epoch in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
        GD_steps += 1
        optimizer.zero_grad()   # do not accumulate gradients every backprop
        
        start = time.time()
        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)


        outputs = model.forward(inputs) #log_ps = model(images)
        loss = criterion(outputs, labels) # loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        if GD_steps % print_every == 0:  #evaluate / test accuracy every given number of GD steps
            # divided by train loader if testing occurs after an entire epoch
            # train_losses.append(running_loss/len(trainloader))
            train_losses.append(running_loss/GD_steps)
            model.eval()
            test_loss = 0
            accuracy = 0      

            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                model.eval()
                #use ALL test data for validation of the model
                for inputs, labels in testloader:
                    outputs = model(inputs)
                    test_loss += criterion(outputs, labels)
                    
                    ps = torch.exp(outputs)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                    
            test_losses.append(test_loss/len(testloader))
            accuracy_values.append(accuracy/len(testloader))

            print(f"Gradient Steps: {GD_steps}.."
                    f"Epoch {epoch+1}/{epochs}.."
                    f"Avg Train loss: {running_loss/print_every:.3f}.."
                    f"Avg Test loss: {test_loss/len(testloader):.3f}.."
                    f"Avg Test accuracy: {accuracy/len(testloader):.3f}")
    # set model back to train mode
    #print(f"Device = {device}; Time per epoch: {(time.time() - start):.3f} seconds",
    #          f"Training Loss: {running_loss/len(trainloader):.3f}.. ")
    model.train()   


Gradient Steps: 5..Epoch 1/1..Avg Train loss: 0.672..Avg Test loss: 0.460..Avg Test accuracy: 0.745
Gradient Steps: 10..Epoch 1/1..Avg Train loss: 1.059..Avg Test loss: 0.273..Avg Test accuracy: 0.929


KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.plot(accuracy_values, label='Accuracy')
plt.legend(frameon=False)

You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [ ]:
## TODO: Use a pretrained model to classify the cat and dog images